## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)



In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Punta Arenas,CL,-53.15,-70.92,57.20,47,75,47.20,light rain
1,1,Kapaa,US,22.08,-159.32,78.80,65,75,6.93,broken clouds
2,2,Peniche,PT,39.36,-9.38,60.01,81,93,4.00,overcast clouds
3,3,Vila Franca Do Campo,PT,37.72,-25.43,59.00,72,20,23.04,few clouds
4,4,Ushuaia,AR,-54.80,-68.30,50.00,61,75,28.86,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_data_df=city_data_df.loc[(city_data_df["Max Temp"]>=min_temp)&(city_data_df["Max Temp"]<=max_temp)]
preferred_city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
1,1,Kapaa,US,22.08,-159.32,78.80,65,75,6.93,broken clouds
8,8,Atar,MR,20.52,-13.05,75.18,20,1,4.12,clear sky
13,13,Lorengau,PG,-2.02,147.27,80.31,81,73,4.70,broken clouds
14,14,Rikitea,PF,-23.12,-134.97,75.67,76,1,10.00,clear sky
17,17,Bubaque,GW,11.28,-15.83,82.24,56,99,8.99,overcast clouds
20,20,Vaini,TO,-21.20,-175.20,77.00,78,20,4.70,few clouds
25,25,Cockburn Town,TC,21.46,-71.14,80.60,61,20,6.93,few clouds
29,29,Guerrero Negro,MX,27.98,-114.06,75.18,20,24,6.17,few clouds
34,34,Mahebourg,MU,-20.41,57.70,77.00,83,40,12.75,shower rain
37,37,Puerto Ayora,EC,-0.74,-90.35,77.00,65,75,4.70,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_city_data_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_city_data_df.dropna()
clean_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Kapaa,US,78.80,broken clouds,22.08,-159.32,
8,Atar,MR,75.18,clear sky,20.52,-13.05,
13,Lorengau,PG,80.31,broken clouds,-2.02,147.27,
14,Rikitea,PF,75.67,clear sky,-23.12,-134.97,
17,Bubaque,GW,82.24,overcast clouds,11.28,-15.83,
20,Vaini,TO,77.00,few clouds,-21.20,-175.20,
25,Cockburn Town,TC,80.60,few clouds,21.46,-71.14,
29,Guerrero Negro,MX,75.18,few clouds,27.98,-114.06,
34,Mahebourg,MU,77.00,shower rain,-20.41,57.70,
37,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping")
    
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [44]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()


In [48]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
hotel_clean_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
hotel_clean_df=hotel_clean_df[hotel_clean_df["Hotel Name"].notna()]
hotel_clean_df.to_csv(output_data_file,index_label="City_ID")
hotel_clean_df.head()


,City_ID,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,1,Kapaa,US,78.80,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
1,8,Atar,MR,75.18,clear sky,20.52,-13.05,Oumou Elghoura
2,13,Lorengau,PG,80.31,broken clouds,-2.02,147.27,Lorengau Harbourside Hotel
3,14,Rikitea,PF,75.67,clear sky,-23.12,-134.97,Pension Maro'i
4,17,Bubaque,GW,82.24,overcast clouds,11.28,-15.83,"Casa Dora, Bubaque"


In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))